## Sensitivity analysis - finite differences


In [1]:
%%capture
## compile PyRoss for this notebook
import os
owd = os.getcwd()
os.chdir('../../')
%run setup.py install
os.chdir(owd)

In [2]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import pyross
import time 
from scipy.misc import derivative

In [24]:
M = 2                # the population has two age groups
N = 5e4              # and this is the total population

# parameters for generating synthetic trajectory 
beta  = 0.02         # infection rate
gIa   = 1./7         # recovery rate of asymptomatic infectives
gIs   = 1./7         # recovery rate of asymptomatic infectives
alpha = 0.2          # fraction of asymptomatic infectives
fsa   = 0.8          # the self-isolation parameter

# set the age structure
fi = np.array([0.25, 0.75])  # fraction of population in age age group
Ni = N*fi

# set the contact structure
C = np.array([[18., 9.], 
              [3., 12.]]) 
# C_ij = number of people group from group i that an individual from group j meets per day 

# set up initial condition
Ia0 = np.array([10, 10])  # each age group has asymptomatic infectives
Is0 = np.array([2, 2])  # and also symptomatic infectives
R0  = np.array([0, 0])    # there are no recovered individuals initially
S0  = Ni - (Ia0 + Is0 + R0)

Tf = 100
Nf = Tf+1

def contactMatrix(t):
    return C

parameters = {'alpha':alpha, 'beta':beta, 'gIa':gIa, 'gIs':gIs,'fsa':fsa}
X = np.load('sto_traj_fix.npy').astype('float')
x = (X/N)

steps = 5
estimator = pyross.inference.SIR(parameters, M, fi, int(N), steps)

# use faster ODE methods to speed up inference 
estimator.set_lyapunov_method('RK2')
estimator.set_det_method('euler')

In [25]:
# logP of correct parameters 
start_time = time.time() 
parameters = {'alpha':alpha, 'beta':beta, 'gIa':gIa, 'gIs':gIs,'fsa':fsa}
logp = estimator.obtain_minus_log_p(parameters, x, Tf, Nf,
                                    contactMatrix, tangent=True)
end_time = time.time()
print(logp) 
print(end_time - start_time)

-4286.325041188613
0.014685869216918945


In [26]:
# the names of the parameters to be inferred 
keys = ['alpha', 'beta', 'gIa', 'gIs']

# Define the prior (log normal prior around guess of parameter with defined std. deviation)
alpha_g = 0.25
beta_g = 0.04
gIa_g = 0.1
gIs_g = 0.1

# initial guess 
guess = np.array([alpha_g, beta_g, gIa_g, gIs_g])  

# error bars on the initial guess 
alpha_std = 0.2
beta_std = 0.1
gIa_std = 0.2
gIs_std = 0.2
stds = np.array([alpha_std, beta_std , gIa_std, gIs_std])

eps = 1e-4
# bounds on the parameters 
bounds = np.array([(eps, 0.8), (eps, 0.2), (eps, 0.6), (eps, 0.6)]) 

# Stopping criterion for minimisation (realtive change in function value)
ftol = 1e-6  
start_time = time.time() 
params = estimator.infer_parameters(keys, guess, stds, bounds, x, 
                                    Tf, Nf, 
                                    contactMatrix, 
                                    tangent=True, 
                                    global_max_iter=20,
                                    local_max_iter=200, 
                                    global_ftol_factor=1e1, 
                                    enable_global=False,
                                    ftol=ftol, 
                                    verbose=True)
end_time = time.time()

print(params) # best guess 
print(end_time - start_time)

Optimal value (local minimisation):  -4295.532243996349
[0.20952031 0.02008036 0.14791327 0.14231912]
1.0980720520019531


In [6]:
x0 = x[0,:]

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

## Tangent space

In [7]:
start = time.time()
FIM_tan = estimator.FIM(keys,params, x0, Tf, Nf,
                    contactMatrix, dx=1e-10, tangent=True)
end = time.time()
print(end-start)

22.090938568115234


In [8]:
## FIM_tan positive definite? 
is_pos_def(FIM_tan)

True

In [9]:
## 'expected' Bayesian credible intervals, 
## order: np.concatenate(x0,epi_params)
expected_BCI_tan = np.sqrt(np.diagonal(np.linalg.inv(FIM_tan)))
expected_BCI_tan

array([0.01420718, 0.02202296, 0.00021697, 0.00024023, 0.00026591,
       0.00028671, 0.01942767, 0.00063902, 0.0149506 , 0.00578613])

## Manifold

In [10]:
start = time.time()
FIM_mfd = estimator.FIM(keys,params, x0, Tf, Nf,
                    contactMatrix, dx=1e-10, tangent=False)
end = time.time()
print(end-start)

90.1551296710968


In [11]:
## FIM_tan positive definite? 
is_pos_def(FIM_mfd)

True

In [12]:
## 'expected' Bayesian credible intervals, 
## order: np.concatenate(x0,epi_params)
expected_BCI_mfd = np.sqrt(np.diagonal(np.linalg.inv(FIM_mfd)))
expected_BCI_mfd

array([0.01404174, 0.02132621, 0.00026352, 0.00028354, 0.0003283 ,
       0.00035327, 0.01998453, 0.00062269, 0.01568876, 0.00598668])

## Sensitivity measure from Komogorov paper (tangent)

In [13]:
from scipy.linalg import eig

In [14]:
evals, evecs = eig(FIM_tan)

In [15]:
L = np.diag(np.real(evals))
C = evecs 

In [16]:
S_ij = np.sqrt(L)@C
S2_ij = S_ij**2
S2_j = np.sum(S2_ij, axis=0)
Norm = np.sum(S2_j)
T_j = np.divide(S2_j,Norm)

In [17]:
## Sensitivity measure
T_j

array([0.00326135, 0.00178729, 0.0014629 , 0.00297122, 0.00097864,
       0.00396068, 0.00689749, 0.89999071, 0.06737493, 0.0113148 ])

In [18]:
## Eigenvalues 
np.real(evals)

array([3.74726774e+09, 3.26697291e+07, 1.64916715e+07, 9.07541595e+06,
       9.01974035e+06, 4.80151076e+04, 9.26003742e+03, 5.44010502e+03,
       2.26449861e+03, 1.76240671e+03])

### Aside: Hessian investigation


In [27]:
import numdifftools as nd

In [28]:
params

array([0.20952031, 0.02008036, 0.14791327, 0.14231912])

In [29]:
def logP(params):
    params = {'alpha':params[0], 'beta':params[1], 'gIa':params[2],
             'gIs':params[3], 'fsa': fsa}
    logp = estimator.obtain_minus_log_p(params, x, Tf, Nf,
                                    contactMatrix, tangent=True)
    return logp


In [30]:
## Try hessian with numdifftools
hess_ = nd.Hessian(logP, step=1.e-4, method='central')
hess = hess_(params) ## this is potentially easier than our Hessian

hess_pyross = estimator.compute_hessian(keys, params, guess, 
                                        stds, x, Tf, Nf, 
                                        contactMatrix, eps=1.e-4)
sigma = np.sqrt(np.diagonal(np.linalg.inv(hess)))
sigma_pyross = np.sqrt(np.diagonal(np.linalg.inv(hess_pyross)))

## What's the difference? 
rel_err = np.divide(np.subtract(sigma,sigma_pyross), sigma_pyross)

np.set_printoptions(formatter={'float_kind':'{:.8f}'.format})
    
    
print('Numdifftool:   ',sigma)
print('NumHessian:    ',sigma_pyross)
print('RelError:      ',rel_err)
print('\n')
print('Autograd:      ',[0.00547942, 0.00010017, 0.00390985, 0.0012171 ] )


Numdifftool:    [0.00391204 0.00007097 0.00279738 0.00086475]
NumHessian:     [0.00540851 0.00010293 0.00387417 0.00120003]
RelError:       [-0.27668745 -0.31043768 -0.27793999 -0.27939063]


Autograd:       [0.00547942, 0.00010017, 0.00390985, 0.0012171]


### Verdict: Our Hessian works better than numdifftool ... verified by autograd hessian (Python + JAX)